# 🎮 Battle City RL - DreamerV3 Training

This notebook trains a **DreamerV3** agent to play **Battle City (NES)**.

## Setup
1. **Runtime**: Go to `Runtime` > `Change runtime type` > Select **T4 GPU** (or better).
2. **Run All**: Execute cells sequentially (Ctrl+F9).

### 1. Clone Repository
First, we clone the project from GitHub.

In [ ]:
# Clone the repo
!git clone https://github.com/Nikolay1221/x.git battle_city

# Change to project directory
%cd battle_city

# Show files
!ls -la

### 2. Install Dependencies
Install NES emulator, DreamerV3, and JAX (GPU).

In [ ]:
# System dependencies for NES emulator
!apt-get update -qq
!apt-get install -y -qq freeglut3-dev xvfb

# DreamerV3 and Embodied (Official from GitHub)
!pip install -q git+https://github.com/danijar/embodied.git
!pip install -q git+https://github.com/danijar/dreamerv3.git

# Environment & Utils
!pip install -q gym gymnasium nes-py opencv-python matplotlib

# JAX (GPU) - Colab compatible
!pip install -q -U "jax[cuda12_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

print("\n✅ Installation complete!")

### 3. Verify Setup
Check that all required files and imports work.

In [ ]:
import os

# Check files
required = ['BattleCity.nes', 'battle_city_env.py', 'dreamer_wrapper.py', 'train_dreamer.py']
missing = [f for f in required if not os.path.exists(f)]

if missing:
    print(f"❌ Missing: {missing}")
else:
    print("✅ All files present!")

# Test imports
try:
    import jax
    print(f"✅ JAX devices: {jax.devices()}")
except Exception as e:
    print(f"❌ JAX error: {e}")

try:
    import dreamerv3
    print("✅ DreamerV3 imported")
except Exception as e:
    print(f"❌ DreamerV3 error: {e}")

### 4. Launch TensorBoard (Optional)
Monitor training metrics in real-time.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/dreamer

### 5. Start Training 🚀
This launches DreamerV3 training. 
- Training logs go to `logs/dreamer`
- Expect initial slow progress, then rapid improvement
- Check TensorBoard for metrics

In [ ]:
# Use virtual display (no monitor in Colab)
!Xvfb :1 -screen 0 1024x768x24 &
import os
os.environ['DISPLAY'] = ':1'

# Launch training
!python train_dreamer.py